In [4]:
#%% IMPORT LIBRARIES

#import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import gpboost as gpb #https://github.com/fabsig/GPBoost/tree/master; https://github.com/fabsig/GPBoost/blob/master/examples/python-guide/GPBoost_algorithm.py
import time

#%% Set seaborn theme and random state
sns.set_theme(context='paper', style = 'ticks',font_scale=1.2)
rs = 621

OSError: dlopen(/Users/jilliangreene/grad_school/sentinel3_ch4_model/venv/lib/python3.12/site-packages/gpboost/lib_gpboost.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <0755E224-5C72-303A-AA12-10FB17E0E6F6> /Users/jilliangreene/grad_school/sentinel3_ch4_model/venv/lib/python3.12/site-packages/gpboost/lib_gpboost.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

In [ ]:
#%% READ DATA 
#read Landsat 8 and 9 data
landsat = pd.read_csv("data/raw/JRC50/landsat_micorps_joined_jrc50_lagos.csv")
landsat = landsat.dropna(axis=0)

#read sentinel-2 data
s2 = pd.read_csv("data/raw/JRC50/sentinel_micorps_joined_jrc50_lagos.csv")
s2 = s2.dropna(axis=0)

#shuffle the rows of the dataframes
landsat = landsat.sample(frac=1).reset_index(drop=True)
s2 = s2.sample(frac=1).reset_index(drop=True)

#%%rename Landsat band 5 to B5B8A and Sentinel-2 Band 8A to B5B8A
#these are the NIR bands, which are named differently for Landsat and S2
landsat = landsat.rename(columns={"B5": "B5B8A"})
s2 = s2.rename(columns={"B8A": "B5B8A"})

#%% Combine Landsat-S2 data for plotting/mapping
landsat_s2 = pd.concat([landsat, s2], axis=0, join='inner')

#%% CREATE NEW COLUMNS (Band ratios)

from  itertools import combinations

#Landsat
landsat_bands = landsat.loc[:, ['B1', 'B2', 'B3', 'B4', 'B5B8A']]
band_ratios_calc = {f'{a}/{b}': landsat_bands[a].div(landsat[b]) for a, b in combinations(landsat_bands.columns, 2)}
landsat_band_ratios = pd.concat(band_ratios_calc, axis=1)
landsat = pd.concat([landsat,landsat_band_ratios], axis=1)

#Sentinel-2
s2_bands = s2.loc[:, ['B1', 'B2', 'B3', 'B4', 'B5B8A']]
band_ratios_s2_calc = {f'{a}/{b}': s2_bands[a].div(s2[b]) for a, b in combinations(s2_bands.columns, 2)}
s2_band_ratios = pd.concat(band_ratios_s2_calc, axis=1)
s2 = pd.concat([s2,s2_band_ratios], axis=1)

#%%Convert dates to datetime
#add column 't' to indicate days since start of sample data to use in Gaussian process model
landsat['Date_Sampled'] = pd.to_datetime(landsat['Date_Sampled'], format = '%m/%d/%Y %H:%M:%S')
landsat['image_date'] = pd.to_datetime(landsat['image_date'], format = '%m/%d/%Y %H:%M:%S')
landsat['year'] = landsat['image_date'].dt.year
landsat['month'] = landsat['image_date'].dt.month
landsat['t'] = (landsat['image_date']-pd.to_datetime('04/30/2013 00:00:00', format = '%m/%d/%Y %H:%M:%S')).dt.days

s2['Date_Sampled'] = pd.to_datetime(s2['Date_Sampled'], format = '%m/%d/%Y %H:%M:%S')
s2['image_date'] = pd.to_datetime(s2['image_date'], format = '%m/%d/%Y %H:%M:%S')
s2['year'] = s2['image_date'].dt.year
s2['month'] = s2['image_date'].dt.month
s2['t'] = (s2['image_date']-pd.to_datetime('04/30/2013 00:00:00', format = '%m/%d/%Y %H:%M:%S')).dt.days


#%% Load LAGOS-NE IWS data, merged with dataframe
iws_lulc = pd.read_csv("data/raw/lagos_ne/LAGOSNE_iws_lulc105.csv")

iws_lulc['iws_nlcd2011_pct_dev'] = iws_lulc['iws_nlcd2011_pct_21'] + \
    iws_lulc['iws_nlcd2011_pct_22'] + iws_lulc['iws_nlcd2011_pct_23'] + \
        iws_lulc['iws_nlcd2011_pct_24']
        
iws_lulc['iws_nlcd2011_pct_ag'] =  iws_lulc['iws_nlcd2011_pct_81'] +  iws_lulc['iws_nlcd2011_pct_82']

iws_human = iws_lulc[['lagoslakeid','iws_nlcd2011_pct_dev', 'iws_nlcd2011_pct_ag']]

#left_join landsat with iws_human

landsat = landsat.merge(iws_human, on="lagoslakeid")
s2 = s2.merge(iws_human, on="lagoslakeid")

#%% Based on uneven lake-samples per HUC, combine based on geography

huc4_group = {707: '0402-0403-0707', 402:'0402-0403-0707', 
              403: '0402-0403-0707', 405: '0405', 406: '0406', 407: '0407-0408', 
              408: '0407-0408', 409: '0409-0410', 410: '0409-0410'}

landsat['HUC4_Group'] = landsat['HU4'].map(huc4_group)
s2['HUC4_Group'] = s2['HU4'].map(huc4_group)

#%% Get feature names

#concatenate dataframes of original band values and ratios
features = pd.concat([landsat_bands, landsat_band_ratios], axis=1, join="inner")

feature_names = list(features.columns)
#feature_names.extend(['iws_nlcd2011_pct_dev','iws_nlcd2011_pct_ag'])

#%% Find matching landsat and sentinel dates with secchi data. These matches will be used as a final test dataset
#allowing comparision between performance of sentinel and Sentinel-2 models

s2_landsat_match = landsat.merge(s2, right_on=['image_date', 'STORETID', 'Date_Sampled'], left_on=['image_date', 'STORETID','Date_Sampled'])
#s2_landsat_match.to_csv('data/output/s2_landsat_secchi_match.csv')

#%% create training and testing sets, where test set is coincident S2/Landsat data
# Note that coincident data is from 2019-2022
id_cols = s2_landsat_match[['image_date', 'STORETID', 'Date_Sampled']]

#get test set based on matching image date, storetid, and sample date for secchi data
landsat_test_set = landsat.merge(id_cols, how="inner", on=['image_date', 'STORETID', 'Date_Sampled']).sort_values(['STORETID','Date_Sampled'])
s2_test_set = s2.merge(id_cols, how="inner", on=['image_date', 'STORETID', 'Date_Sampled']).sort_values(['STORETID','Date_Sampled'])

#get training set based on data not in test_set
landsat_train_set = landsat.merge(landsat_test_set.drop_duplicates(), how='left', indicator=True)
landsat_train_set = landsat_train_set[landsat_train_set['_merge'] == 'left_only']

s2_train_set = s2.merge(s2_test_set.drop_duplicates(), how='left', indicator=True)
s2_train_set = s2_train_set[s2_train_set['_merge'] == 'left_only']

# Cconcatenate training datasets, keep only common columns
full_landsat_s2_train_set = pd.concat([landsat_train_set, s2_train_set], axis=0, join='inner')

#%% Print training and testing data
columns_to_print = ['STORETID','Lat','Lon','lagoslakeid','satellite','image_date',
         'B1','B2', 'B3', 'B4', 'B5B8A',
         'B1/B2','B1/B3','B1/B4',
         'B1/B5B8A','B2/B3','B2/B4','B2/B5B8A',
         'B3/B4','B3/B5B8A','B4/B5B8A',
         'Date_Sampled', 'secchi_m','HU4','HU6','HU8','HU12','HUC4_Group',
         'iws_nlcd2011_pct_dev',
         'iws_nlcd2011_pct_ag',
         't', 'month', 'year']

full_landsat_s2_train_set.to_csv('data/processed/Combined_LandsatSentinel2_Secchi_trainingdata.csv',
                         columns=columns_to_print, index=False)
landsat_test_set.to_csv('data/processed/Landsat_Secchi_testdata.csv',
                         columns=columns_to_print, index=False)
s2_test_set.to_csv('data/processed/Sentinel2_Secchi_testdata.csv',
                         columns=columns_to_print, index=False)

#%% Define model evaluation function 
#Eval stats selected from Pontius "Metrics that make a difference" book.
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def evaluate(test_labels, predictions,  mod_name):
    mean_dev = np.mean(predictions) - np.mean(test_labels)
    mae = mean_absolute_error(test_labels, predictions)
    pearson = stats.pearsonr(predictions, test_labels.ravel())[0]
    slope, i = np.polyfit(test_labels,predictions,1)
    r2 = r2_score(test_labels, predictions)
    rmse = mean_squared_error(test_labels, predictions, squared=False)
    print('Model Performance')
    print('Mean Deviation: {:0.3f} m'.format(np.mean(mean_dev)))
    print('Mean Absolute Error: {:0.3f} m'.format(mae))
    print('RMSE: {:0.3f} m'.format(rmse))
    print('Slope = {:0.3f}'.format(slope))
    print('Pearson correlation = {:0.3f}'.format(pearson))
    print('r2 = {:0.3f}'.format(r2))
    values = [mean_dev, mae, rmse, slope, pearson, r2]
    metrics = pd.DataFrame(values, 
                           columns = ['statistic'])
    metrics.index = ['MeanDeviation','MAE','RMSE', 'Slope',
               'Pearson', 'r2']
    metrics.to_csv('data/results/model_statistics_' + mod_name +'.csv', index=True)


#%% Function "prep_data"
#Prep data for use in the models - give the data the original dataframe

def prep_data(training_df, testing_df):
    x_train = training_df[feature_names].to_numpy()
    x_test = testing_df[feature_names].to_numpy()
    
    #response training and testing
    y_train = training_df['secchi_m'].to_numpy()
    y_test = testing_df['secchi_m'].to_numpy()
    
    lake = training_df['lagoslakeid'].to_numpy()
    lake_test = testing_df['lagoslakeid'].to_numpy()
    
    huc = training_df['HU4'].to_numpy()
    huc_test = testing_df['HU4'].to_numpy()
    
    huc4_grp = training_df['HUC4_Group'].to_numpy()
    huc4_grp_test = testing_df['HUC4_Group'].to_numpy()
    
    year = training_df['year'].to_numpy()
    year_test = testing_df['year'].to_numpy()
    
    month = training_df['month'].to_numpy()
    month_test = testing_df['month'].to_numpy()
    
    t_train = training_df['t'].to_numpy()
    t_test = testing_df['t'].to_numpy()
    
    return x_train, x_test, y_train, y_test, \
        lake, lake_test, huc, huc_test, huc4_grp, huc4_grp_test, \
            year, year_test, month, month_test, t_train, t_test

#%% GPBOOST - Grid search tuning and cv boost function

# arguments = split type, experiment name (e.g. Landsat, year splits)
# returns a dictionary of best parameters and model performance metrics

param_grid = {'learning_rate': [0.01, 0.05, 0.1, 0.5, 1], 
              'min_data_in_leaf': [5,10,100,500],
              'max_depth': [-1, 1, 2, 3, 5, 10],
              'lambda_l2': [0,1,10, 100]}

    #simple parameter grid for testing
    # param_grid = {'learning_rate': [0.5], 
    #               'min_data_in_leaf': [10],
    #               'max_depth': [1,2],
    #               'lambda_l2': [0,1]}

def model_tune_crossval(splits, par_grid, experiment_name):
    
    other_params = {'num_leaves': 2**10, 'verbose': 0}
    
    #set up panel model with random effects = lake
    #temporal random slopes = days since 4/30/2013 (start of dataset)
    gp_model = gpb.GPModel(group_data=pd.DataFrame({'lake':lake}), 
                           likelihood="gaussian",
                           group_rand_coef_data=t_train, 
                           ind_effect_group_rand_coef=[1])

    data_train = gpb.Dataset(data=x_train, label=y_train)

    #initialize grid search (note mae will be listed as 'l1-mean' in output)
    opt_pars = gpb.grid_search_tune_parameters(param_grid=par_grid, 
                                                 params=other_params,
                                                 num_boost_round=1000,
                                                 num_try_random=None, 
                                                 folds = splits, 
                                                 seed=81388,
                                                 train_set=data_train, 
                                                 gp_model=gp_model,
                                                 use_gp_model_for_validation=True, 
                                                 verbose_eval=0,
                                                 early_stopping_rounds=10,
                                                 metric=["mae","rmse"])                                             
    
    # Get best number of boosting iterations
    cvbst = gpb.cv(params=opt_pars['best_params'], 
                   train_set=data_train,
                   gp_model=gp_model, 
                   use_gp_model_for_validation=True,
                   num_boost_round=1000, 
                   early_stopping_rounds=10,
                   folds = splits, 
                   show_stdv=False, 
                   seed=22587, 
                   metric=["mae","rmse"])
    
    best_iter_index = np.argmin(cvbst['l1-mean']) #cv boost best iteration
    
    best_pars = opt_pars['best_params']
    
    #add model results to dictionary
    best_pars['iterations'] = best_iter_index +1
    best_pars['mae-mean'] = cvbst['l1-mean'][best_iter_index]
    best_pars['mae-stdv'] = cvbst['l1-stdv'][best_iter_index]
    best_pars['rmse-mean'] = cvbst['rmse-mean'][best_iter_index]
    best_pars['rmse-stdv'] = cvbst['rmse-stdv'][best_iter_index]
    best_pars['experiment'] =  experiment_name
    
    return(best_pars)
    

#%% LANDSAT: split data by holdout dataset of coincident Landsat-S2
#must call this function prior to calling the cross val function to train model
x_train, x_test, y_train, y_test, \
    lake, lake_test, huc, huc_test, \
        huc4_grp, huc4_grp_test, \
            year, year_test, month, month_test, \
                t_train, t_test = prep_data(landsat_train_set, landsat_test_set)

#%% LANDSAT- YEAR SPLITS, GRID SEARCH TUNE PARAMETERS
from sklearn.model_selection import LeaveOneGroupOut

#split by year
logo = LeaveOneGroupOut()
splits_year = list(logo.split(x_train, y_train, groups=year))

#dictionary of best simulation results is returned (parameters and error stats)
results_yr_fold = model_tune_crossval(splits_year, param_grid, "Landsat-Year-Folds")

#%% LANDSAT- HUC4 Group SPLITS, GRID SEARCH TUNE PARAMETERS
from sklearn.model_selection import LeaveOneGroupOut

logo = LeaveOneGroupOut()
splits_huc = list(logo.split(x_train, y_train, groups=huc4_grp))

#dictionary of best simulation results is returned (parameters and error stats)
results_huc_fold = model_tune_crossval(splits_huc, param_grid, "Landsat-HUC-Folds")

#%% LANDSAT- LAKE (lagoslakeid) SPLITS, w/ 10-fold, GRID SEARCH TUNE PARAMETERS
from sklearn.model_selection import GroupKFold

group_kfold = GroupKFold(n_splits=10)
splits_lake = list(group_kfold.split(x_train, y_train, groups=lake))

#dictionary of best simulation results is returned (parameters and error stats)
results_lake_10fold = model_tune_crossval(splits_lake, param_grid, "Landsat-Lake-Folds")

#%% LANDSAT: SIMPLE 10-FOLD CROSS VALIDATION (NO GROUPS)
from sklearn.model_selection import KFold
         
kfold = KFold(n_splits = 10, random_state = 100, shuffle = True)

#dictionary of best simulation results is returned (parameters and error stats)
results_10fold = model_tune_crossval(kfold, param_grid, "Landsat-10-Folds")

#START RUN HERE

#%% LANDSAT: GRADIENT TREE-BOOSTING with NO GROUP VARIABLE (NO RANDOM EFFECTS)


# param_grid = {'learning_rate': [0.5, 0.1], 
#               'min_data_in_leaf': [10,100],
#               'max_depth': [1,2],
#               'lambda_l2': [0,1]}

param_grid = {'learning_rate': [0.01, 0.05, 0.1, 0.5, 1], 
              'min_data_in_leaf': [5,10,100,500],
              'max_depth': [-1, 1, 2, 3, 5, 10],
              'lambda_l2': [0,1,10, 100]}
other_params = {'num_leaves': 2**10, 'verbose': 0}
    
kfold = KFold(n_splits = 10, random_state = 100, shuffle = True)

data_train = gpb.Dataset(data=x_train, label=y_train)

#initialize grid search (note mae will be listed as 'l1-mean' in output)
opt_pars_gbm = gpb.grid_search_tune_parameters(gp_model=None,
                                           param_grid=param_grid,
                                           params=other_params,
                                           num_boost_round=1000,
                                           num_try_random=None, 
                                           folds = kfold, 
                                           seed=81388,
                                           train_set=data_train, 
                                           use_gp_model_for_validation=False, 
                                           verbose_eval=0,
                                           early_stopping_rounds=10,
                                           metric=["mae", "rmse"])                                             
                                               
    # Get best number of boosting iterations
cvbst = gpb.cv(params=opt_pars_gbm['best_params'], 
               train_set=data_train,
               num_boost_round=1000, 
               early_stopping_rounds=10,
               folds = kfold, 
               show_stdv=False, 
               seed=22587, 
               metric=["mae", "rmse"])

best_iter_index_gbm = np.argmin(cvbst['l1-mean']) #cv boost best iteration

best_pars_gbm = opt_pars_gbm['best_params']

#add model results to dictionary
best_pars_gbm['iterations'] = best_iter_index_gbm +1
best_pars_gbm['mae-mean'] = cvbst['l1-mean'][best_iter_index_gbm]
best_pars_gbm['mae-stdv'] = cvbst['l1-stdv'][best_iter_index_gbm]
best_pars_gbm['rmse-mean'] = cvbst['rmse-mean'][best_iter_index_gbm]
best_pars_gbm['rmse-stdv'] = cvbst['rmse-stdv'][best_iter_index_gbm]
best_pars_gbm['experiment'] =  "Landsat-GBM-FixedEffects-10fold"

#%% LANDSAT GPB: COMBINE cross validation dictionary results, write to csv
import collections

dict_list = [results_yr_fold, results_huc_fold,
                         results_lake_10fold, results_10fold,
                         best_pars_gbm]

landsat_results_gridsearch = collections.defaultdict(list)
for d in dict_list:
    for k, v in d.items():  # d.items() in Python 3+
        landsat_results_gridsearch[k].append(v)

pd.DataFrame.from_dict(data=landsat_results_gridsearch, orient='index') \
    .to_csv('data/results/landsat_gridsearch_results.csv', header=False)
    

#%% LANDSAT ONE MORE TUNE (10-fold) to refine

param_grid_2 = {'learning_rate': [0.005, 0.01, 0.02], 
              'min_data_in_leaf': [3,5,7],
              'max_depth': [-1, 10, 15],
              'lambda_l2': [80,100,120]} 

kfold = KFold(n_splits = 10, random_state = 100, shuffle = True)

#dictionary of best simulation results is returned (parameters and error stats)
results_10fold_finaltune = model_tune_crossval(kfold, param_grid_2, "Landsat-10-Folds-FinalTune")


#%% LANDSAT GPB FINAL MODEL TUNE NUM ROUNDS

#best parameters are from best pars of regular k-fold, final tuning
best_pars = {'lambda_l2': 80, 
                  'learning_rate': 0.02, 
                  'max_depth': -1, 
                  'min_data_in_leaf': 5}

data_train = gpb.Dataset(data=x_train, label=y_train)

gp_model = gpb.GPModel(group_data=pd.DataFrame({'lake':lake}), 
                       likelihood="gaussian",
                       group_rand_coef_data=t_train, 
                       ind_effect_group_rand_coef=[1])
#redo boost rounds to get best number

cvbst = gpb.cv(params=best_pars, 
               train_set=data_train,
               gp_model=gp_model, 
               use_gp_model_for_validation=True,
               num_boost_round=3000, 
               early_stopping_rounds=10,
               folds = kfold, 
               show_stdv=False, 
               seed=22587, 
               metric=["mae","rmse"])

best_iter_index_gpb = np.argmin(cvbst['l1-mean']) #cv boost best iteration

#%%
gpb_landsat = gpb.train(params=best_pars, 
                        train_set=data_train,  
                        gp_model=gp_model,
                        num_boost_round=best_iter_index_gpb)
#gp_model.summary() # Estimated random effects model

# Save model
gpb_landsat.save_model('data/output/model_gpb_landsat.json')
# Load from file and make predictions again
#bst_loaded = gpb.Booster(model_file = 'model.json')

#other way to save the model with joblib
import joblib
joblib.dump(gpb_landsat, 'data/output/model_gpb_landsat.pkl')
# load model
#bst_loaded = joblib.load('data/output/model_gpb_landsat.pkl')

#%% LANDSAT GPB TRAINED MODEL PREDICTIONS ON HOLDOUT DATASET
# Predict response variable (pred_latent=False)
# pred_resp['response_mean']: mean predictions of the response variable 
# which combines predictions from the tree ensemble and the random effects
# pred_resp['response_var']: predictive (co-)variances (if predict_var=True)
pred_resp = gpb_landsat.predict(data=x_test, 
                                        group_data_pred=pd.DataFrame({'lake':lake_test}),
                                        group_rand_coef_data_pred = t_test,
                                        predict_var=False,
                                        pred_latent=False)

#extract predictions
preds_landsat_testset = pred_resp['response_mean']

#%% Plot prediction on test dataset (Landsat model)
#Test dataset = coincident Landsat+Sentinel data
#Using Landsat reflectance values
#https://stackoverflow.com/questions/19064772/visualization-of-scatter-plots-with-overlapping-points-in-matplotlib

values = np.vstack([y_test.ravel(), preds_landsat_testset.ravel()])
kernel = stats.gaussian_kde(values, bw_method= 0.5)(values)

plt.scatter(y_test, preds_landsat_testset, s=20, c=kernel,cmap='viridis')
plt.axline((0,0), (13,13), linewidth=1, color='black')
plt.axis((0,13,0,13))
plt.xlabel("In-situ SDD (m)")
plt.ylabel("Predicted SDD (m)")
plt.colorbar()
plt.savefig('data/figures/gpboost/GPBoost_Landsat_coincident_data_holdout'+ time.strftime("%Y%m%d-%H%M%S")+ '.jpg', dpi=300)

evaluate(y_test, preds_landsat_testset, 'GPB_Landsat_testset')

#%% GPB Print final predictions on test set

landsat_test_set["secchi_pred_m"] = preds_landsat_testset

landsat_preds_out = landsat_test_set[["lagoslakeid", "STORETID", "Lat", "Lon",
                                     "satellite", "pathrow", "image_date", "Date_Sampled",
                                     "secchi_m", "secchi_pred_m"]]

landsat_preds_out.to_csv("data/results/GPB_landsat_testset_predictions.csv", index=False)    


#%% LANDSAT GBM (NO RANDOM EFFECTS): TRAIN BEST MODEL

gbm = gpb.train(params=best_pars_gbm, 
                        train_set=data_train,  
                        num_boost_round=best_iter_index_gbm)

pred_resp_gbm = gbm.predict(data=x_test, predict_var=False, pred_latent=False)

evaluate_gbm = evaluate(y_test, pred_resp_gbm, 'GBM_Landsat')

values = np.vstack([y_test.ravel(), pred_resp_gbm.ravel()])
kernel = stats.gaussian_kde(values, bw_method= 0.5)(values)

plt.scatter(y_test, pred_resp_gbm, s=20, c=kernel, cmap='viridis')
plt.axline((0,0), (13,13), linewidth=1, color='black')
plt.axis((0,13,0,13))
plt.xlabel("In-situ SDD (m)")
plt.ylabel("Predicted SDD (m)")
plt.colorbar()
plt.savefig('data/figures/gbm/GBM_Landsat_Coincident_holdout.jpg', dpi=300)

#%% GBM (NO RANDOM EFFECTS) Print final predictions

landsat_test_set["secchi_pred_m"] = pred_resp_gbm

landsat_preds_out_gbm = landsat_test_set[["lagoslakeid", "STORETID", "Lat", "Lon",
                                     "satellite", "pathrow", "image_date", "Date_Sampled",
                                     "secchi_m", "secchi_pred_m"]]

landsat_preds_out_gbm.to_csv("data/results/GBM_landsat_testset_predictions.csv", index=False)

#%% LANDSAT GPB: Plot SHAP 
#https://github.com/AidanCooper/shap-analysis-guide/blob/main/analysis.ipynb
import shap

Xpd = landsat_train_set[feature_names]

shap_values = shap.Explainer(gpb_landsat).shap_values(Xpd)
shap.summary_plot(shap_values, Xpd, show=False)
plt.savefig('data/figures/gpboost/shap_landsat.jpg', dpi=300)

#%%TRAIN FINAL LANDSAT MODEL AND APPLY TO ALL REFLECTANCE DATA

best_pars = {'lambda_l2': 80, 
                  'learning_rate': 0.02, 
                  'max_depth': -1, 
                  'min_data_in_leaf': 5}

groups = landsat["lagoslakeid"].to_numpy()
time = landsat["t"].to_numpy()

gpb_model_landsat_final = gpb.GPModel(group_data=pd.DataFrame({'lake': groups}), 
                       likelihood="gaussian",
                       group_rand_coef_data= time, 
                       ind_effect_group_rand_coef=[1])

data_train_final = gpb.Dataset(data=landsat[feature_names].to_numpy(), 
                         label=landsat["secchi_m"].to_numpy())

gpb_landsat_final_trained = gpb.train(params=best_pars, 
                        train_set=data_train_final,  
                        gp_model=gpb_model_landsat_final,
                        num_boost_round=1146)

gpb_model_landsat_final.summary() # Estimated random effects model